In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')     # 한글 폰트
import matplotlib as mpl
mpl.rc('axes', unicode_minus=False)           # 유니코드 "-" sign

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.datasets import reuters

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical


In [2]:
# global constants and hyper-parameters
MY_SAMPLE = 2947
NUM_CLASS = 46          # Classification class
MY_NUM_WORDS = 2000     # number of words in dictionary


MY_HIDDEN = 512         # 
MY_DROPOUT = 0.5        # Dropout rate : temporarily dropout given rate of cell's outputs to "0" : like Regularization

MY_EPOCH = 10
MY_BATCH = 64

In [3]:
labels = ['cocoa','grain','veg-oil','earn','acq','wheat','copper',
          'housing','money-supply','coffee','sugar','trade','reserves', 
          'ship','cotton','carcass','crude','nat-gas','cpi','money-fx',
          'interest','gnp','meal-feed','alum','oilseed','gold','tin',
          'strategic-metal','livestock','retail','ipi','iron-steel',
          'rubber','heat','jobs','lei','bop','zinc','orange',
          'pet- chem','dlr','gas','silver','wpi','hog','lead']

In [4]:
def show_shape():
  print('\n== DB SHAPE INFO ==')
  print('X_train shape = ', X_train.shape)
  print('X_test shape = ', X_test.shape)
  print('Y_train shape = ', Y_train.shape)
  print('Y_test shape = ', Y_test.shape) 
  print()

In [5]:
(X_train, Y_train), (X_test, Y_test) = reuters.load_data(num_words = MY_NUM_WORDS, test_split = 0.3)

show_shape()

2121728/2110848 [==============================] - 0s 0us/step

== DB SHAPE INFO ==
X_train shape =  (7859,)
X_test shape =  (3369,)
Y_train shape =  (7859,)
Y_test shape =  (3369,)



In [6]:
# statistics on how many articles per category in the train DB
# numpy unique is useful in this case
print('\n== TRAIN DATA CONTENT INFO ==')
unique, counts = np.unique(Y_train, return_counts = True)
for i in range(len(unique)):
  print(unique[i], labels[i], "=", counts[i])


== TRAIN DATA CONTENT INFO ==
0 cocoa = 50
1 grain = 378
2 veg-oil = 66
3 earn = 2769
4 acq = 1701
5 wheat = 14
6 copper = 39
7 housing = 15
8 money-supply = 126
9 coffee = 93
10 sugar = 114
11 trade = 337
12 reserves = 40
13 ship = 149
14 cotton = 18
15 carcass = 19
16 crude = 387
17 nat-gas = 33
18 cpi = 59
19 money-fx = 475
20 interest = 238
21 gnp = 91
22 meal-feed = 10
23 alum = 36
24 oilseed = 56
25 gold = 77
26 tin = 18
27 strategic-metal = 13
28 livestock = 43
29 retail = 19
30 ipi = 38
31 iron-steel = 34
32 rubber = 30
33 heat = 9
34 jobs = 43
35 lei = 10
36 bop = 46
37 zinc = 17
38 orange = 16
39 pet- chem = 20
40 dlr = 32
41 gas = 28
42 silver = 10
43 wpi = 19
44 hog = 10
45 lead = 14


In [8]:
# show a sample data in its raw format
print('\n== SAMPLE ARTICLE (RAW) ==')
print("article #", MY_SAMPLE)
print("category", Y_train[MY_SAMPLE], labels[Y_train[MY_SAMPLE]])
print("number of words", len(X_train[MY_SAMPLE]))
print(X_train[MY_SAMPLE])

# python dictionary: word -> index
# zero index is not used
word_to_id = reuters.get_word_index()
print('\n== DICTIONARY INFO ==')
print("There are", len(word_to_id) + 1, "words in the dictionary.")
print('The index of "the" is', word_to_id['the'])


== SAMPLE ARTICLE (RAW) ==
article # 2947
category 4 acq
number of words 61
[1, 2, 1229, 81, 8, 16, 515, 25, 270, 5, 4, 2, 1229, 111, 267, 7, 73, 2, 2, 7, 108, 13, 80, 1448, 28, 365, 12, 11, 15, 1986, 2, 69, 158, 18, 1296, 1275, 7, 2, 1627, 2, 2, 4, 393, 374, 1229, 323, 5, 2, 1229, 7, 2, 9, 25, 2, 473, 936, 4, 49, 8, 17, 12]
565248/550378 [==============================] - 0s 0us/step

== DICTIONARY INFO ==
There are 30980 words in the dictionary.
The index of "the" is 1


In [9]:
# python dictionary: index -> word
# this is the opposite to word_to_id dictionary
id_to_word = {}
for key, value in word_to_id.items():
  id_to_word[value] = key

In [10]:
def decoding():
  decoded = []
  for i in X_train[MY_SAMPLE]:
    word = id_to_word.get(i - 3, "???")
    decoded.append(word)

  print('\n== SAMPLE ARTICLE (DECODED) ==')
  print(" ".join(decoded))
  
decoding()
print("category", Y_train[MY_SAMPLE], labels[Y_train[MY_SAMPLE]])


== SAMPLE ARTICLE (DECODED) ==
??? ??? telephone corp said it completed its acquisition of the ??? telephone co based in new ??? ??? in exchange for stock valued at 26 3 mln dlrs enterprises ??? about 16 000 access lines in ??? county ??? ??? the third operating telephone subsidiary of ??? telephone in ??? and its ??? largest overall the company said reuter 3
category 4 acq


In [11]:
# we will NOT do padding (as in movie review classification)
# instead we will do tokenization for the inputs
# we get a numpy array of size MY_NUM_WORDS for each input 
# the entries are integer counts 
# the resulting matrix is very big
from keras.preprocessing.text import Tokenizer
# for i in range(10):
#   print(len(X_train[i]))

Tok = Tokenizer(num_words = MY_NUM_WORDS)

print('before:', X_train[0])
print('number before:', len(X_train[0]))
X_train = Tok.sequences_to_matrix(X_train, mode = 'count')
print('after:', X_train[0])
print('number after:', len(X_train[0]))

X_test = Tok.sequences_to_matrix(X_test, mode = 'count')

before: [1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 2, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 2, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]
number before: 87
after: [0. 1. 4. ... 0. 0. 0.]
number after: 2000


In [12]:
# Tok = Tokenizer(num_words = MY_NUM_WORDS)
# X_train = Tok.sequences_to_matrix(X_train, mode = 'count')
# X_test = Tok.sequences_to_matrix(X_test, mode = 'count’)

print('\n== SAMPLE ARTICLE (TOKENIZED INPUT) ==')
sample = X_train[MY_SAMPLE]
print(*sample, sep = ' ')
print("Array size:", len(sample))
print("Sum of entries:", np.sum(sample))


== SAMPLE ARTICLE (TOKENIZED INPUT) ==
0.0 1.0 11.0 0.0 3.0 2.0 0.0 4.0 2.0 1.0 0.0 1.0 2.0 1.0 0.0 1.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 2.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0

In [13]:
# output reshaping using one-hot encoding
# from keras.utils import to_categorical
Y_train = to_categorical(Y_train, NUM_CLASS)
Y_test = to_categorical(Y_test, NUM_CLASS)

print('\n== SAMPLE ARTICLE (1-HOT ENCODING OUTPUT) ==')
sample = Y_train[MY_SAMPLE]
print(sample)
print("Array size:", len(sample))

show_shape()



== SAMPLE ARTICLE (1-HOT ENCODING OUTPUT) ==
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Array size: 46

== DB SHAPE INFO ==
X_train shape =  (7859, 2000)
X_test shape =  (3369, 2000)
Y_train shape =  (7859, 46)
Y_test shape =  (3369, 46)



In [14]:
###############################
# MODEL BUILDING AND TRAINING #
###############################
# build a keras sequential model of our DNN
# softmax is needed for multi-class classification
model = Sequential()
model.add(Dense(units=MY_HIDDEN, input_shape = (MY_NUM_WORDS,)))
# model.add(Dense(MY_HIDDEN, input_shape = (MY_NUM_WORDS,)))  => have to give Tuple, eg (MY_NUM_WORDS,)
model.add(Activation('relu'))       # later comment this out or use others, 
model.add(Dropout(MY_DROPOUT))
model.add(Dense(NUM_CLASS))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               1024512   
                                                                 
 activation (Activation)     (None, 512)               0         
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 46)                23598     
                                                                 
 activation_1 (Activation)   (None, 46)                0         
                                                                 
Total params: 1,048,110
Trainable params: 1,048,110
Non-trainable params: 0
_________________________________________________________________


In [15]:
# prediction using the model
# shape needs to change from (2000,) to (1, 2000)
def ask_question():
  sample = X_train[MY_SAMPLE]
  sample = sample.reshape(1, sample.shape[0])
  pred = model.predict(sample, verbose = 0)
  guess = np.argmax(pred)
  answer = np.argmax(Y_train[MY_SAMPLE])
  
  print('\n== SAMPLE QUESTION ==')
  print("My guess for sample article:", guess, labels[guess])
  print("The answer is:", answer, labels[answer]) 
  print()

ask_question()


== SAMPLE QUESTION ==
My guess for sample article: 41 gas
The answer is: 4 acq



In [ ]:
# model training and saving
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X_train, Y_train, validation_data = (X_test, Y_test), epochs = MY_EPOCH, batch_size = MY_BATCH, verbose = 1)
model.save('chap2.h5')

In [ ]:
####################
# MODEL EVALUATION #
####################
# evaluate the model and calculate loss and accuracy
score = model.evaluate(X_test, Y_test, verbose = 1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

pred = model.predict(X_test)
print('Prediction for the first news: ', pred[0])
print(np.argmax(pred[0])) 
print('Answer: ', Y_test[0])
print(labels[np.argmax(Y_test[0])]) 


# ask_question()